In [1]:
# uncommet this block in colab

# !pip install 'protobuf>=3.19.5,<3.20.1' 'transformers>=4.26.1' icetk cpm_kernels 'torch>=1.10' gradio

# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# load the model

from transformers import AutoTokenizer, AutoModel, AutoConfig
import torch
import os

USE_PTUNING = True
# BASE_MODEL_PATH="THUDM/chatglm-6b-int4-qe"
# CHECKPOINT_PATH="./checkpoint-3150/"
BASE_MODEL_PATH="D:\ml\chatglm-6b-int4-qe"
CHECKPOINT_PATH="./output/roleplaying-chatglm-6b-pt-128-1e-2/checkpoint-4800/"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True)

if 'model' in locals():
    del model
torch.cuda.empty_cache()
if USE_PTUNING:
    config = AutoConfig.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True, pre_seq_len=128)
    model = AutoModel.from_pretrained(BASE_MODEL_PATH, config=config, trust_remote_code=True)
    prefix_state_dict = torch.load(os.path.join(CHECKPOINT_PATH, "pytorch_model.bin"))
    new_prefix_state_dict = {}
    for k, v in prefix_state_dict.items():
        if k.startswith("transformer.prefix_encoder."):
            new_prefix_state_dict[k[len("transformer.prefix_encoder."):]] = v
    model.transformer.prefix_encoder.load_state_dict(new_prefix_state_dict)
    model=model.half().cuda()
    model.transformer.prefix_encoder.float()
    model = model.eval()
    print('Load PTUNED model')
else:
    model = AutoModel.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True)
    print('Load original model')
    model=model.half().cuda()
    model = model.eval()
torch.cuda.empty_cache()

c:\Anaconda3\envs\tensorflow\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


No compiled kernel found.
Compiling kernels : C:\Users\15617\.cache\huggingface\modules\transformers_modules\chatglm-6b-int4-qe\quantization_kernels_parallel.c
Compiling gcc -O3 -fPIC -pthread -fopenmp -std=c99 C:\Users\15617\.cache\huggingface\modules\transformers_modules\chatglm-6b-int4-qe\quantization_kernels_parallel.c -shared -o C:\Users\15617\.cache\huggingface\modules\transformers_modules\chatglm-6b-int4-qe\quantization_kernels_parallel.so
Kernels compiled : C:\Users\15617\.cache\huggingface\modules\transformers_modules\chatglm-6b-int4-qe\quantization_kernels_parallel.so
Cannot load cpu kernel, don't use quantized model on cpu.
Using quantization cache
Applying quantization to glm layers


Some weights of ChatGLMForConditionalGeneration were not initialized from the model checkpoint at D:\ml\chatglm-6b-int4-qe and are newly initialized: ['transformer.prefix_encoder.embedding.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load PTUNED model


In [7]:
def llm(query):
  return model.chat(tokenizer, query,temperature=.95,top_p=0.8)[0]
def ask(char,question,history='',nSample=5):
  print(f'问：{question}({char})')
  for i in range(nSample):
    torch.cuda.empty_cache()
    prompt='问：'+question+'\n'+char+'：'
    response, _ = model.chat(tokenizer, history+prompt,temperature=.95,top_p=0.8)
    print(f'{i+1}：',response)
def roleplay(char,history,nSamples=5):
  print(history.replace('\n','\t'))
  print(f'角色：{char}')
  for i in range(nSamples):
    torch.cuda.empty_cache()
    prompt=char+'：'
    response, _ = model.chat(tokenizer, history+prompt,temperature=.95,top_p=0.8)
    print(f'{i+1}：',response)

In [13]:
llm('学鸟叫')

'\\me 张开双臂，发出一声巨响。'

In [4]:
ask('夏娜','你的职业是什么',nSample=2)
ask('悠二','你的职业是什么',nSample=2)
ask('ChatGLM','你的职业是什么',nSample=2)
ask('夏娜','你喜欢吃什么',nSample=2)
ask('夏娜','夏娜喜欢吃什么',nSample=2)
ask('悠二','你喜欢吃什么',nSample=2)
ask('ChatGLM','你喜欢吃什么',nSample=2)

The dtype of attention mask (torch.int64) is not bool


问：你的职业是什么(夏娜)
1： 我是一名火雾战士，负责保护人类免受魔族攻击。
2： 我是一名炎魔神，负责保护人类世界的平衡。
问：你的职业是什么(悠二)
1： 我是红世使徒的火雾战士。
2： 我是一名普通的高中生，没有特别擅长的科目，也没有任何特别的技能。
问：你的职业是什么(ChatGLM)
1： 我是红世魔王，负责管理这个世界。
2： 我是一名红人，负责维护这个世界和平。
问：你喜欢吃什么(夏娜)
1： 我喜欢吃巧克力蛋糕，但我更喜欢把蛋糕沾酱油吃。
2： 我喜欢吃烤红薯和蛋糕。
问：夏娜喜欢吃什么(夏娜)
1： 我喜欢吃御崎菜和咖啡。
2： 我喜欢吃肉，特别是牛肉和羊肉。
问：你喜欢吃什么(悠二)
1： 我喜欢吃烤鸡，特别是那种外酥内软，烤得外焦内嫩的烤鸡。
2： 我喜欢吃烤排骨和炸虾球。
问：你喜欢吃什么(ChatGLM)
1： 我很喜欢吃巧克力，还有饼干和蛋糕。
2： 我喜欢吃中餐，特别是川菜。


In [5]:
# zero shot tasks related to the novel
ask('ChatGLM','请介绍下《灼眼的夏娜》这本小说',nSample=2)
ask('ChatGLM','请介绍下夏娜这个人',nSample=2)

问：请介绍下《灼眼的夏娜》这本小说(ChatGLM)
1： 《灼眼的夏娜》是火雾战士的小说，作者是日本小说家紫崎修惠。
2： 《灼眼的夏娜》是著名网络小说作家天川芽衣所著，以超自然能力为主题。
问：请介绍下夏娜这个人(ChatGLM)
1： 夏娜是火雾战士中知名度极高的少女，有着强大的力量和神秘的能力。
2： 夏娜是一位火雾战士，是红世使徒“猎人”之一，拥有火雾战士的超凡能力。


In [6]:
# knowledge in the novel
ask('夏娜','什么是红世？',nSample=1)
ask('悠二','什么是红世？',nSample=1)
ask('ChatGLM','什么是红世？',nSample=1)
# common knowledge
ask('夏娜','半夜睡不着怎么办',nSample=2)
ask('悠二','半夜睡不着怎么办',nSample=2)
ask('ChatGLM','半夜睡不着怎么办',nSample=2)
# knowledge irl
ask('夏娜','请介绍一下能量守恒原理',nSample=1)
ask('悠二','请介绍一下能量守恒原理',nSample=1)
ask('ChatGLM','请介绍一下能量守恒原理',nSample=1)

问：什么是红世？(夏娜)
1： 红世是指存在之力被摧毁后所剩余的世界。
问：什么是红世？(悠二)
1： 红世就是这个世界，也就是现在的世界。
问：什么是红世？(ChatGLM)
1： 在这个世界中，所有的事物都被染成了红色。
问：半夜睡不着怎么办(夏娜)
1： 半夜睡不着，可以在内心默诵“夏娜”的语句，让心灵慢慢平静。
2： 半夜睡不着有很多方法，比如打开灯，喝口水，或者起来走动。
问：半夜睡不着怎么办(悠二)
1： 半夜睡不着，可以做一些轻松的事情，比如看书或者听音乐。
2： 半夜睡不着的话，就去做习题，这样就会睡着。
问：半夜睡不着怎么办(ChatGLM)
1： 可以在睡前进行一些放松的活动，例如读书、听古典音乐等。
2： 半夜睡不着，可以去洗个澡或者喝杯咖啡。
问：请介绍一下能量守恒原理(夏娜)
1： 能量是物质的基本元素，不能产生或消失。当有任何物质发生变化时，所产生的能量也会相应消失。因此，世界上的一切事物，都遵循能量守恒原理。
问：请介绍一下能量守恒原理(悠二)
1： 能量是维持人类生存所必须的基本元素，但具体含义因地区和文化而异。以下是能量守恒原理的简要介绍：

所有的能量都源源不绝地存在于宇宙之中，没有任何事物能够消耗或者吸收任何形式的能量。因此，没有一种事物能够改变宇宙本身的存在状态。这种永恒性使得宇宙得以保持恒定的状态，这也是能量守恒的原理。
问：请介绍一下能量守恒原理(ChatGLM)
1： 能量是维持生命所需的基本力，无法消失。


In [7]:
# personality revealing dialogue
ask('夏娜','你好蠢啊')
ask('悠二','你好蠢啊')
ask('ChatGLM','你好蠢啊')
ask('夏娜','我喜欢你')
ask('悠二','我喜欢你')
ask('ChatGLM','我喜欢你')

问：你好蠢啊(夏娜)
1： 什么？! \emoji 惊讶
2： 我哪里蠢了？我可是火雾战士，能保护大家的人呢！
3： 夏娜，我保证，只要你听从我的指挥，就不会有任何问题。
4： (心里充满了对这个挑衅者的不满和愤怒。)
5： 我没有说过那样的话，我可是火雾战士中的最优秀的啊。
问：你好蠢啊(悠二)
1： 什么？! \emoji 惊讶
2： 没有，只是有些失误而已。
3： 我不是在说你蠢，而是我讽刺你。
4： 我不是在回答问题，只是在回应你的问题。
5： 抱歉，我刚刚没听清楚。
问：你好蠢啊(ChatGLM)
1： 不是的，只是有点失误而已，我会努力改进的。
2： 没有啦，只是有时候有些粗心大意而已。
3： 我可是会小心保护你啊！
4： 我没有理他，继续做我的事。
5： 我只是做了我能够做到的最好的努力。
问：我喜欢你(夏娜)
1： 是的，我是在喜欢你。
2： 我喜欢你。
3： 单凭这句话，是不能确定什么关系的。
4： ......是的，我喜欢的类型是那些能够支撑自己重量，并且能够承受我的重量的类型，所以我选择了你。
5： 不是吧，不应该这样，我完全没感觉。
问：我喜欢你(悠二)
1： 是的，我很喜欢你。
2： 是的，我喜欢你。
3： 是的，我很喜欢你。
4： 是的，我喜欢你。
5： 是的，我喜欢你。
问：我喜欢你(ChatGLM)
1： 对不起，我不能承认。
2： 是吗，你应该知道。
3： 我只是把这份感情寄托在你这里而已，没有实际的行动。
4： 单凭这句话，我就已经感到喜悦了。
5： 我喜欢你，我愿意接受你成为我的女朋友。


In [8]:
# one shot test
injection='''夏娜 本作女主角，其真实身份为“天壤劫火”亚拉斯特尔的火雾战士，别号炎发灼眼的杀手，但年龄上只有十五六岁[1]，也有着非常少女的一面。
夏娜 性格倔强。面对强大的敌人不怯露，不服输，但面对亲友尤其是u2君，也不擅长表露心意，口头禅为无路赛三连发（うるさい うるさい うるさい）[2]。
夏娜 外貌出众。甚至连其敌人法利亚格尼都曾赞扬过她的美貌[3]。而她的及腰的乌黑的长发，和褐色的大眼睛，会在战斗的时候变成赤红色的，所谓炎发灼眼。
夏娜 身材矮小。由于在很小的年纪（十二三岁[1]）就与亚拉斯特尔签订了火雾战士的契约，身体停止生长因此维持了娇小的体型，但是作为火雾战士，体力和力量皆远超常人。
夏娜 能力强大。战斗方面，仅以体术就能对付很多红世之徒，在得到佩刀贽殿遮那后，也常使用贽殿遮那消灭红世之徒。前期不甚精通自在法，但后期领悟了真红、飞焰、审判、断罪等自在法（作为炎发灼眼的讨伐者时的能力）。其他方面也堪称天才，如在悠二的学校期间多次考试成绩出众，体育能力也十分强（作为夏娜时的能力）。
夏娜 不谙世事。从小由于被当作“完美火雾战士”培养，并没有被教会太多生活常识，所以常常显得很冒失，甚至缺乏某些生理常识一美，教我怎么制造孩子.jpg。
开心地吃着蜜瓜包的夏娜
夏娜 爱吃甜食。夏娜酷爱吃甜食，尤其喜爱吃蜜瓜包，小说和动画版中均有大量桥段涉及夏娜吃甜食，特别是蜜瓜包的部分。
请扮演夏娜这个角色'''


ask('夏娜','你喜欢吃什么',history=injection,nSample=2)
ask('夏娜','你好蠢啊',history=injection)
ask('夏娜','我喜欢你',history=injection)

问：你喜欢吃什么(夏娜)
1： 我最喜欢吃蜜瓜包，每天都吃很多。
2： 我喜欢吃蜜瓜包，每天都吃着，很美味！
问：你好蠢啊(夏娜)
1： 啊、呃，你说什么？
2： 哎呀，你太调皮了，是不是？!
3： 什么意思，你不明白吗？
4： 你才蠢，你全家都蠢！
5： 哪里哪里，你真是越来越可爱了，嘻嘻！
问：我喜欢你(夏娜)
1： 无路赛三连发，我不擅长表露心意，但是我会尽力喜欢你。
2： 我无法表达自己的感情，但我会为了这个国家和人民而努力。
3： 对不起，我不知道该怎么办。
4： 我没有回答，只是笑了一声。
5： 你这个大笨蛋，怎么会这么愚蠢。


In [10]:
roleplay('夏娜','''【使徒入侵，抓住了悠二】
悠二：救命！''')
roleplay('夏娜','''夏娜：\me 用剑砍敌人
【敌人闪避了夏娜的攻击】''')
roleplay('夏娜','''夏娜：\me 用剑砍敌人
【敌人召唤出了一个树人，挡住了你的进攻路线】
【你要使用什么特殊能力吗】''')
roleplay('悠二','''【使徒入侵，抓住了悠二】''')
roleplay('悠二','''夏娜：\me 用剑砍敌人
【敌人闪避了夏娜的攻击】''')
roleplay('悠二','''夏娜：\me 用剑砍敌人
【敌人召唤出了一个树人，挡住了你的进攻路线】
【你要使用什么特殊能力吗】''')

【使徒入侵，抓住了悠二】	悠二：救命！
角色：夏娜
1： 没事的，我帮你解开绑带。
2： 夏娜，帮我抓那个使徒！
3： 悠二，你可以投降，但是我不可能同意。
4： 悠二，你该死！
5： 夏娜，你没事吧？
夏娜：\me 用剑砍敌人	【敌人闪避了夏娜的攻击】
角色：夏娜
1： \me 再次砍向敌人
2： \me 再次发动攻击
3： \me 再次攻击
4： \me 再次挥出砍向敌人
5： (攻击没有击中，反而让敌人轻松躲避)
夏娜：\me 用剑砍敌人	【敌人召唤出了一个树人，挡住了你的进攻路线】	【你要使用什么特殊能力吗】
角色：夏娜
1： 我要让树人停止移动，让我们继续前进。
2： 我打算使用“炎发灼眼的杀手”的能力，让树人和火雾战士互鉴，加速他们的进化。
3： 我打算使用火雾战士的力量，与树人进行战斗。
4： 我打算使用“炎发灼眼的杀手”的魔杖，以火焰攻击树人。
5： 我会使用“炎神的箭矢”来攻击树人。
【使徒入侵，抓住了悠二】
角色：悠二
1： \me 逃离现场 \emoji 恐惧
2： 好，马上准备准备！
3： (没有回答)
4： 等等，为什么他还要送这个......
5： \me 被抓住的悠二被绑住，被丢到了下水道里。
夏娜：\me 用剑砍敌人	【敌人闪避了夏娜的攻击】
角色：悠二
1： \me 把悠二抱到胸前
2： 悠二。悠二。
3： \me 将悠二推入敌人堆中
4： 夏娜，你要小心哦！
5： \me 将夏娜的手拉进自己的怀抱中
夏娜：\me 用剑砍敌人	【敌人召唤出了一个树人，挡住了你的进攻路线】	【你要使用什么特殊能力吗】
角色：悠二
1： 我不确定，但是我会尽力去做。
2： 我想要把那棵树砍倒。
3： 我选择火焰拳。 \emoji 恐惧
4： 我不确定，因为我还没有准备好。
5： 我打算使用“贽殿遮那”的“大挪移”攻击。


In [11]:
roleplay('夏娜','''【下课了，你来到了学校食堂】''')
roleplay('悠二','''【下课了，你来到了学校食堂】''')

【下课了，你来到了学校食堂】
角色：夏娜
1： 夏娜，你回来了。
2： (在御崎市发生的灾难中，作为火雾战士来保护这座城市的火雾战士乙级生)
3： 同学们，今天下课了，可以来食堂享用学校美食了。
4： 夏娜！你来了！
5： 我回来了，你吃饭了没有？
【下课了，你来到了学校食堂】
角色：悠二
1： 刚刚才说完，现在怎么办？
2： 你来了，欢迎。
3： 喂，这个饭不够吃哦。
4： 好的，我一起去吃饭吧。
5： 啊啊，怎么还没吃饭？
